## Preparación del modelo

In [ ]:
#EXPLICAR UN POCO LOS PASOS

In [2]:
#https://www.kaggle.com/code/paola311/clasificaci-n-de-im-genes-cnn
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
import keras
from keras import layers

dir_general = 'C:/Users/nuria/Downloads/TFG/data'

dir_train = os.path.join(dir_general, 'train')
dir_validation = os.path.join(dir_general, 'val')
dir_test = os.path.join(dir_general, 'test')

# Preprocesamiento de imágenes
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen=ImageDataGenerator(rescale=1./255)

#Iterador que recorre el directorio de imágenes
train_generator = train_datagen.flow_from_directory(
    dir_train,
    target_size=(150, 150),
    batch_size=20, #ir probando potencias de 2
    class_mode='binary')

validation_generator = validation_datagen.flow_from_directory(
    dir_validation,
    target_size=(150, 150),
    batch_size=20, #lomas grande posible que no cause problemas de memoria 
    class_mode='binary')

test_generator = test_datagen.flow_from_directory(
    dir_test,
    target_size=(150, 150),
    batch_size=20, #lomas grande posible que no cause problemas de memoria 
    class_mode='binary')

Found 5216 images belonging to 2 classes.
Found 16 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [7]:
'''#Ejemplo para ver como se accede a las imágenes y etiquetas
for imagenes, etiquetas in train_generator:
    print("Imágenes:", imagenes[1]) #si no pongo [1] sale de 20 en 20 porque el batch size es 20
    print("Etiquetas:", etiquetas[1])'''

'#Ejemplo para ver como se accede a las imágenes y etiquetas\nfor imagenes, etiquetas in train_generator:\n    print("Imágenes:", imagenes[1]) #si no pongo [1] sale de 20 en 20 porque el batch size es 20\n    print("Etiquetas:", etiquetas[1])'

In [3]:
input_shape=(150,150,3)

model = keras.Sequential( #funcion establecer arquitectura(simple...)
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"), 
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        #se necesitan mas capas
        layers.Dropout(0.5), #probar otros valores (este es muy alto)
        layers.Dense(1, activation="sigmoid"), #una unica neurina, sigmoide
    ]
)

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 74, 74, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 72, 72, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 36, 36, 64)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 82944)             0         
                                                                 
 dropout (Dropout)           (None, 82944)             0

## Entrenar el modelo

In [9]:
#https://stackoverflow.com/questions/43345909/when-using-mectrics-in-model-compile-in-keras-report-valueerror-unknown-metr
#En esta version de keras no hay un f1 disponible por lo que he de crearlo yo misma

In [28]:
y_test=test_generator.labels
y_pred=model.predict(test_generator)

32/32 [==============================] - 7s 226ms/step


In [30]:
#otra forma de hacerlo
np.round(y_pred)

array([[1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],

In [41]:
'''
se está aplicando un umbral para convertir las probabilidades de predicción en una clasificación binaria. En el caso de un problema de clasificación binaria, como determinar si una radiografía muestra neumonía o no, el modelo neuronal puede proporcionar una probabilidad como salida, donde un valor cercano a 1 indica alta confianza en la clase positiva (neumonía) y un valor cercano a 0 indica alta confianza en la clase negativa (no neumonía).

Aplicar un umbral de 0.5 significa que cualquier probabilidad mayor que 0.5 se considerará como predicción de la clase positiva, y cualquier probabilidad menor o igual a 0.5 se considerará como predicción de la clase negativa. Esto es comúnmente usado en problemas de clasificación binaria para convertir las probabilidades continuas en etiquetas de clase discretas.
'''
y_pred=y_pred>0.5

In [11]:
#creo que borrar
#no sé si esto está bien
#y_pred_labels = np.argmax(y_pred, axis=1)

In [72]:
#FUNCIONA
from keras import backend as K

def f1_score(y_true, y_pred):
    precision = K.sum(y_true * K.round(y_pred)) / (K.sum(K.round(y_pred)) + K.epsilon())
    recall = K.sum(y_true * K.round(y_pred)) / (K.sum(y_true) + K.epsilon())
    return 2 * ((precision * recall) / (precision + recall + K.epsilon()))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[f1_score])

model.fit(train_generator, epochs=10, batch_size=32, validation_data=validation_generator)


Epoch 1/10
 73/261 [=======>......................] - ETA: 1:01 - loss: 0.2865 - f1_score: 0.9216

KeyboardInterrupt: 

In [42]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
import numpy as np

def metricas(y_test, y_pred):
    exactitud = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    sensibilidad = recall_score(y_test, y_pred)
    puntaje = f1_score(y_test, y_pred)
    return (exactitud,precision,sensibilidad,puntaje)

In [67]:
#prueba
batch_size = 20 #si quito batch size, no me identifica el de train
epochs = 2

model.compile(loss="binary_crossentropy", optimizer="adam", metrics=[metrica_f1]) #cambias loss

model.fit(train_generator, batch_size=batch_size, epochs=epochs, validation_data=validation_generator) #callback earlyestopping

Epoch 1/2


ValueError: in user code:

    File "C:\Users\nuria\anaconda3\envs\tf\lib\site-packages\keras\engine\training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\nuria\AppData\Local\Temp\ipykernel_13556\1478855858.py", line 5, in metrica_f1  *
        prec = metrics.Precision()(y_true, y_pred)
    File "C:\Users\nuria\anaconda3\envs\tf\lib\site-packages\keras\dtensor\utils.py", line 144, in _wrap_function  **
        init_method(instance, *args, **kwargs)
    File "C:\Users\nuria\anaconda3\envs\tf\lib\site-packages\keras\metrics\metrics.py", line 882, in __init__
        self.true_positives = self.add_weight(
    File "C:\Users\nuria\anaconda3\envs\tf\lib\site-packages\keras\metrics\base_metric.py", line 375, in add_weight
        return super().add_weight(
    File "C:\Users\nuria\anaconda3\envs\tf\lib\site-packages\keras\engine\base_layer.py", line 705, in add_weight
        variable = self._add_variable_with_custom_getter(
    File "C

In [6]:
from keras.callbacks import EarlyStopping

batch_size = 20 #jugar un poco cn este parametro (potencias de 2) (cuanto mayor sea el valor menos tarda)
epochs = 10

model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy","Recall","AUC","Precision"]) #cambias loss

# con callbacks se detiene el entrenamiento si la pérdida en el conjunto de validación no mejora después de 3 épocas (patience)
model.fit(train_generator, batch_size=batch_size, epochs=epochs, validation_data=validation_generator, callbacks=EarlyStopping(monitor='val_loss', patience=3)) #callback earlyestopping

Epoch 1/10
261/261 [==============================] - 152s 575ms/step - loss: 0.1994 - accuracy: 0.9181 - recall: 0.9683 - auc: 0.9653 - precision: 0.9250 - val_loss: 0.5511 - val_accuracy: 0.6875 - val_recall: 1.0000 - val_auc: 1.0000 - val_precision: 0.6154
Epoch 2/10
261/261 [==============================] - 145s 555ms/step - loss: 0.1021 - accuracy: 0.9622 - recall: 0.9770 - auc: 0.9906 - precision: 0.9723 - val_loss: 1.0104 - val_accuracy: 0.6250 - val_recall: 1.0000 - val_auc: 0.9844 - val_precision: 0.5714
Epoch 3/10
261/261 [==============================] - 146s 560ms/step - loss: 0.0847 - accuracy: 0.9703 - recall: 0.9819 - auc: 0.9923 - precision: 0.9781 - val_loss: 0.4271 - val_accuracy: 0.8125 - val_recall: 1.0000 - val_auc: 1.0000 - val_precision: 0.7273
Epoch 4/10
261/261 [==============================] - 144s 551ms/step - loss: 0.0602 - accuracy: 0.9789 - recall: 0.9866 - auc: 0.9958 - precision: 0.9851 - val_loss: 0.0659 - val_accuracy: 1.0000 - val_recall: 1.0000 - 

## Evaluación del modelo 

In [7]:
# DUDA cada vez que se ejecuta sale una cosa distinta, porque?
score = model.evaluate(test_generator, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])
print("Test Recall:", score[2])
print("Test AUC:", score[3])
print("Test Precision:", score[4])

Test loss: 1.6227977275848389
Test accuracy: 0.7323718070983887
Test Recall: 0.9948717951774597
Test AUC: 0.7974413633346558
Test Precision: 0.7016274929046631


In [ ]:
#trabajar con sklearn recall, acuracy
#y_true=test_genrator.labels
#y_pred=modelo.predict(test_generator)

## Trabajando con las métricas

In [ ]:
#matriz de confusión con sklearn? SI
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)